In [2]:
import os               # Импорт модуля для работы с операционной системой (переменные окружения, пути к файлам и т.д.)
import logging          # Импорт модуля для логирования (записи событий и ошибок в программе)
from collections import Counter  # Импорт класса Counter для подсчета частоты элементов в коллекциях
from datetime import datetime, timedelta  # Импорт классов для работы с датой и временем
import requests         # Импорт библиотеки для выполнения HTTP-запросов к веб-серверам
import ast              # Импорт модуля для преобразования строк в структуры данных Python (например, списки, словари)
import psycopg2         # Импорт драйвера для работы с базой данных PostgreSQL

import gspread          # Импорт библиотеки gspread для работы с Google Sheets через API
from psycopg2.extras import execute_batch #для пакетной вставки
from google.oauth2.service_account import Credentials  # Импорт класса Credentials для аутентификации в Google API с использованием сервисного аккаунта

⚡ Перед началом решения задачи **выполните следующую ячейку** - в ней скачиваются нужные файлы логов

In [3]:


log_dir = "logs"                      # Определяем имя директории для хранения лог-файлов
os.makedirs(log_dir, exist_ok=True)   # Создаем директорию для логов, если она не существует (exist_ok предотвращает ошибку если директория уже есть)

!wget -O logs/2026-01-07.log  https://raw.githubusercontent.com/Tolstyakoff/Simulative_FinalPy/main/2026-01-07.log
!wget -O logs/2026-01-11.log  https://raw.githubusercontent.com/Tolstyakoff/Simulative_FinalPy/main/2026-01-07.log
!wget -O logs/2026-01-12.log  https://raw.githubusercontent.com/Tolstyakoff/Simulative_FinalPy/main/2026-01-11.log
!wget -O logs/2026-01-13.log  https://raw.githubusercontent.com/Tolstyakoff/Simulative_FinalPy/main/2026-01-07.log

!wget -O MyKey.json           https://tolstyakoff.ru/media/cases/attachments/MyKey.json

--2026-01-15 17:35:58--  https://raw.githubusercontent.com/Tolstyakoff/Simulative_FinalPy/main/2026-01-07.log
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4105 (4.0K) [text/plain]
Saving to: ‘logs/2026-01-07.log’

logs/2026-01-07.log 100%[===================>]   4.01K  --.-KB/s    in 0s      

2026-01-15 17:35:58 (43.9 MB/s) - ‘logs/2026-01-07.log’ saved [4105/4105]

--2026-01-15 17:35:58--  https://raw.githubusercontent.com/Tolstyakoff/Simulative_FinalPy/main/2026-01-07.log
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4105 

In [4]:
# Во время обработки будет сохраняться лог работы скрипта с отлавливанием всех ошибок и выводом промежуточных стадий
# (например, скачивание началось / скачивание завершилось / заполнение базы началось и т.д., с трекингом времени).
# Лог нужно сохранять в текстовый файл. Файл нужно именовать в соответствии с текущей датой.
# Если в папке с логами уже есть другие логи - их необходимо удалять, оставляем только логи за последние 3 дня.

# Удаляем логи старше 3 дней
log_dir = "logs"  #дублируем

for filename in os.listdir(log_dir):  # Перебираем все файлы в директории логов
    if filename.endswith(".log"):     # Проверяем, является ли файл лог-файлом (имеет расширение .log)
        try:
            # Пытаемся извлечь дату из имени файла (предполагаем формат "ГГГГ-ММ-ДД.log")
            file_date = datetime.strptime(filename.replace(".log", ""), "%Y-%m-%d")   # Преобразуем строку с датой в объект datetime
            if datetime.now() - file_date > timedelta(days=3):                        # Проверяем, старше ли файл 3 дней
                os.remove(os.path.join(log_dir, filename))                            # Удаляем файл, если он старше 3 дней
        except Exception as e: # Обрабатываем возможные ошибки
            print(f"Не удалось обработать лог-файл {filename}: {e}")                  # Выводим сообщение об ошибке в консоль

#####  Используем московское время для имени файла (+3 часа)
import time
from datetime import datetime, timedelta

moscow_time = datetime.now() + timedelta(hours=3)
log_file = os.path.join(log_dir, f"{moscow_time:%Y-%m-%d}.log")


for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.Formatter.converter = lambda *args: time.gmtime(time.time() + 10800)  # +3 часа

logging.basicConfig(
    filename=log_file,
    format='%(asctime)s %(levelname)s: %(message)s',
    level=logging.INFO,
    encoding='utf-8'
)

############# подключаемся по API
api_url = "https://b2b.itresume.ru/api/statistics"  # API адес из ТЗ

# Параметры запроса к API
params = {
    "client": "Skillfactory",                         # Идентификатор клиента
    "client_key": "M2MGWS",                           # Ключ клиента для аутентификации
    "start": "2025-12-31 00:00:00",                   # Начальная дата периода (31 декабря 2025, полночь)
    "end": "2026-01-10 23:59:59"                      # Конечная дата периода (10 января 2026, 23:59:59)
}

logging.info("Подключаемся к API")                    # Вносим лог в файл

try:
    response = requests.get(api_url, params=params)   # Выполняем GET-запрос к API с указанными параметрами
    response.raise_for_status()                       # Проверяем статус ответа: вызывает исключение при HTTP ошибках (4xx, 5xx)
    data = response.json()                            # Парсим JSON-ответ от API в Python-объект (список или словарь)

    logging.info(f"Успешно получено {len(data)} записей: ") #Если все ok, вносим запись в лог

except requests.RequestException as e:               # если с ошибкой, то вносим в лог ощибку и ее код
    logging.error(f"Ошибка при запросе к API: {e}")
    data = []                                        # если ошибка, то data путой массив

############# сохранение полученных данных

processed_data = []

for item in data:                               # Получение параметра passback_params
    raw_passback = item.get("passback_params")

    # Проверка валидности passback_params
    if not raw_passback or not isinstance(raw_passback, str) or raw_passback.strip() == "":
        logging.error(f"passback_params отсутствует или не строка, запись пропущена (user_id={item.get('lti_user_id')})")
        continue                                                                # Пропуск текущей записи при ошибке

    try:                                                                        # Преобразование строки в словарь с проверкой на ошибки
        passback = ast.literal_eval(raw_passback)

        # Обработка поля is_correct
        raw_is_correct = item.get("is_correct")
        if raw_is_correct is None:          # Проверка на отсутствие значения
            is_correct = None
        elif raw_is_correct in (1, True):   # Приведение 1/True к булевому True
            is_correct = True
        elif raw_is_correct in (0, False):  # Приведение 0/False к булевому False
            is_correct = False
        else:                               # Обработка некорректных значений
            logging.error(f"Некорректное поле is_correct: {raw_is_correct}")
            continue                        # Пропуск записи при невалидном is_correct

        # Формирование обработанной записи
        processed_data.append({
            "user_id": item.get("lti_user_id"),                                 # ID пользователя из LTI
            "oauth_consumer_key": passback.get("oauth_consumer_key"),           # Ключ потребителя OAuth
            "lis_result_sourcedid": passback.get("lis_result_sourcedid"),       # Идентификатор результата LTI
            "lis_outcome_service_url": passback.get("lis_outcome_service_url"), # URL сервиса оценок LTI
            "is_correct": is_correct,                                           # Нормализованное значение корректности ответа
            "attempt_type": item.get("attempt_type"),                           # Тип попытки выполнения
            "created_at": item.get("created_at")                                # Время создания записи
        })

    except (ValueError, SyntaxError) as e:                                      # Обработка ошибок парсинга passback_params
        logging.error(f"Некорректный passback_params: {e}, пропуск записи")
        continue                                                                # Пропуск записи при ошибке парсинга
    except Exception as e:  # Обработка любых других ошибок
        logging.error(f"Ошибка при обработке записи: {e}")

# Фиксация результата предобработки
logging.info(f"Предобработка данных завершена, подготовлено {len(processed_data)} записей")



Подключаемся к базе данных и вносим туда данные

In [17]:
############### Запись в облачную базу данных, которой еще нет ##########################

try:                            #Конектор для постгрис
    conn = psycopg2.connect(
        dbname="homework",
        user="tolstyakoff",
        password="qwerty123",
        host="109.248.201.217",
        port="5432"
    )

    cursor = conn.cursor()    #как в лекции
    logging.info("Подключение к базе данных установлено")

    #Сначала удаляем таблицу, если она есть, чтобы не накпливать
    cursor.execute("""
    drop table if exists  grader_attempts
    """)
    # Создание таблицы, если её нет запрос выполнен, но никуда не выводится
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS grader_attempts (
        user_id TEXT,
        oauth_consumer_key TEXT,
        lis_result_sourcedid TEXT,
        lis_outcome_service_url TEXT,
        is_correct BOOLEAN,
        attempt_type TEXT,
        created_at TIMESTAMP
    )
    """)
    conn.commit()
    logging.info("Создана Таблица grader_attempts")


    # SQL запрос для вставки данных
    insert_query = """
      INSERT INTO grader_attempts
      (user_id, oauth_consumer_key, lis_result_sourcedid, lis_outcome_service_url, is_correct, attempt_type, created_at)
      VALUES (%s, %s, %s, %s, %s, %s, %s)
    """

    # Подготовка данных для вставки
    data_to_insert = [
        (
          record['user_id'],
          record['oauth_consumer_key'],
          record['lis_result_sourcedid'],
          record['lis_outcome_service_url'],
          record['is_correct'],
          record['attempt_type'],
          record['created_at']
        )
        for record in processed_data
    ]

    execute_batch(cursor, insert_query, data_to_insert, page_size=1000)

    conn.commit()
    logging.info("Данные успешно загружены ")

except Exception as e:
    logging.error(f"Ошибка работы с базой данных: {e}")

finally:
    if 'cursor' in locals():
        cursor.close()
    if 'conn' in locals():
        conn.close()
    logging.info("Соединение с базой данных закрыто")

In [23]:
########## Быстрый отчет в гугл              ##################
########## С запросом по датам для дашборда  ##################

try:                            #Конектор для постгрис, да еще раз, но так проще
    conn = psycopg2.connect(
        dbname="homework",
        user="tolstyakoff",
        password="qwerty123",
        host="109.248.201.217",
        port="5432"
    )

    cursor = conn.cursor()    #как в лекции
    logging.info("Подключение к базе данных установлено")

    #Сначала удаляем таблицу, если она есть, чтобы не накпливать
    cursor.execute("""
    select distinct to_char(created_at,'YYYY-MM-DD') as "Date",
				count(distinct  user_id) as students,
				count (*) as solve_problems,
				count (case when is_correct then 1 end) as correct_solution,
				count (case when attempt_type = 'run' then 1 end) as run,
				count (case when attempt_type = 'submit' then 1 end) as submit
    from grader_attempts ga
    group by 1
    order by 1
    """)

    conn.commit()
    logging.info("Создана Таблица c отчетом:")

    rows = cursor.fetchall()
    if 'cursor' in locals():
       cursor.close()
    if 'conn' in locals():
       conn.close()
    logging.info("Соединение с базой данных закрыто")

except Exception as e:
    logging.error(f"Ошибка работы с базой данных: {e}")

try:
    # Настройка аутентификации Google API
    scope = ["https://www.googleapis.com/auth/spreadsheets"]                   # Области доступа Google Sheets API
    creds = Credentials.from_service_account_file("MyKey.json", scopes=scope)  # Загрузка учетных данных из файла
    gc = gspread.authorize(creds)                                              # Авторизация в gspread

    # Открываем таблицу и лист
    sheet_id = "1PCsfmyqym4V8SH2BgYhW6QZ-T0FZdW8YPz28Y3mjyU8"
    #sheet = gc.open_by_key(sheet_id).sheet1
    spreadsheet = gc.open_by_key(sheet_id)
    sheet = spreadsheet.worksheet("Stat")

    logging.info("Доступ к Google Sheets получен")       # Успешное логирование

     # Добавляем заголовки (если их еще нет)
    headers = ["Дата", "Студенты", "Всего задач", "Правильно", "Запуски", "Отправки"]
    if sheet.row_count == 0:  # если лист пустой
        sheet.append_row(headers)

    for row in rows:
        sheet_row = [
            row[0],  # Date
            row[1],  # students
            row[2],  # solve_problems
            row[3],  # correct_solution
            row[4],  # run
            row[5]   # submit
        ]
        sheet.append_row(sheet_row)                                # Добавление строки в конец таблицы

    logging.info("Данные успешно загружены в Google Sheets")       # Успешное логирование

except Exception as e:
    logging.error(f"Ошибка при работе с Google Sheets: {e}")       # Логирование ошибок Google Sheets

